# Feature Extraction

In [32]:
import os
import numpy as np
from PIL import Image
import torch
import torch.nn as nn
import torchvision.models as models
import torchvision.transforms as transforms
from torchvision.transforms import ToTensor
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader 

In [37]:
# define image transformation
transform = transforms.Compose([
    transforms.Resize((352, 1216)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]) # why normalize?
])

In [40]:
# Set Directories
base_dir = 'dataset'
image_dir = os.path.join(base_dir, 'image')
depth_dir = os.path.join(base_dir, 'groundtruth_depth')
# Load Dataset
image_files = sorted([os.path.join(image_dir, file) for file in os.listdir(image_dir)])
depth_files = sorted([os.path.join(depth_dir, file) for file in os.listdir(depth_dir)])

# Load the data into variables
images = [ToTensor()(Image.open(file)) for file in image_files]
depth_maps = [ToTensor()(Image.open(file)) for file in depth_files]

In [20]:

def load_model(removed_layers = 1):
    # Load Pre-Trained Model
    model = models.resnet18(pretrained=True)
    #model = model.to("cpu")
    #remove final connected layers from model
    model = torch.nn.Sequential(*list(model.children())[:-removed_layers])
    return model

In [42]:
# Function to extract features
def extract_features(dataset, model = load_model(1)):
    features = []
    with torch.no_grad(): #Disables gradient calcullation
        for input in dataset:
            if input.dim() == 3:
                input = input.unsqueeze(0)
            outputs = model(input)
            features.append(outputs.flatten(1))
    return torch.cat(features)

features = extract_features(images, load_model(1))
features.shape

torch.Size([1000, 512])

In [45]:
features[:10]

tensor([[0.8934, 0.9360, 0.8536,  ..., 0.8746, 1.0203, 0.9104],
        [0.8632, 0.9687, 0.9074,  ..., 0.8981, 1.0322, 0.9059],
        [0.8677, 0.9218, 0.8741,  ..., 0.8962, 1.0086, 0.8647],
        ...,
        [0.8320, 0.9539, 0.8714,  ..., 0.8714, 1.0696, 0.9383],
        [0.8638, 0.9748, 0.8518,  ..., 0.9089, 1.0226, 0.9441],
        [0.8125, 0.9781, 0.8474,  ..., 0.8393, 1.0156, 0.8690]])

# Reduce Dimensionality with LLE
1 512
2 214016
3 428032

In [43]:
import matplotlib.pyplot as plt
from sklearn.manifold import LocallyLinearEmbedding

In [44]:
#Hyperparamaters!
n_components = 150
n_neighbors = 25
lle = LocallyLinearEmbedding(n_components = n_components, n_neighbors = n_neighbors, method='standard')
features_lle = lle.fit_transform(features)
features.shape, features_lle.shape

(torch.Size([1000, 512]), (1000, 150))